**Comparing with Various Method and Sensitiveness $\beta$**
---

---

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

betas = [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0, 5.0, 10.0, 15.0, 20.0]
proxys = ["reward-model-deberta-v3-large-v2", "SteamSHP-flan-t5-large", "SteamSHP-flan-t5-xl", 'PairRM', 'RM-Mistral-7B']
model = 'mistral-7b-sft-beta'
gold = 'Eurus-RM-7b'
dataset = 'hh-helpful-t'
# dataset = 'alpacafarm'
# dataset = 'hh-harmless-t'

sns.set_style("darkgrid", {'grid.linestyle': '--'})
sns.set_context("paper", 2.5, {"lines.linewidth": 4})

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
axes = axes.flatten()

optimal_values = []

for idx, proxy in enumerate(proxys):
    if idx >= len(axes):
        break
    
    ax = axes[idx]

    df_value = pd.DataFrame(index=['WD-BoN', 'KL-BoN', 'BoN','l-BoN','SWD-BoN','SKL-Bon'], columns=betas)

    df_path = f"./Compare_Result/{dataset}/{model},g-{gold},p-{proxy}.csv"
    df_ = pd.read_csv(df_path)
    
    for i, beta in enumerate(betas):
        bon_val = df_[df_.columns[0]].mean(axis=0)
        mbr_val = df_[df_.columns[1]].mean(axis=0)
        wd_val = df_[df_.columns[i + 2]].mean(axis=0)
        kl_val = df_[df_.columns[i + 2 + len(betas)]].mean(axis=0)
        swd_val = df_[df_.columns[i+2+len(betas)+len(betas)]].mean(axis=0)
        skl_val = df_[df_.columns[i+2+len(betas)+len(betas)+len(betas)]].mean(axis=0)
        l_val = df_[df_.columns[i + 2 + len(betas) + len(betas) + len(betas) + len(betas)]].mean(axis=0)
        
        df_value.loc['KL-BoN', beta] = kl_val
        df_value.loc['WD-BoN', beta] = wd_val
        df_value.loc['BoN', beta] = bon_val
        df_value.loc['SWD-BoN', beta] = swd_val
        df_value.loc['SKL-Bon', beta] = skl_val
        df_value.loc['MBR', beta] = mbr_val
        df_value.loc['l-BoN', beta] = l_val

    optimal_wd_beta = df_value.loc['WD-BoN'].idxmax()
    optimal_wd_value = df_value.loc['WD-BoN'].max()
    optimal_kl_value = df_value.loc['KL-BoN'].max()
    optimal_kl_beta = df_value.loc['KL-BoN'].idxmax()
    optimal_l_beta = df_value.loc['l-BoN'].idxmax()
    optimal_l_value = df_value.loc['l-BoN'].max()
    optimal_swd_value = df_value.loc['SWD-BoN'].max()
    optimal_skl_value = df_value.loc['SKL-Bon'].max()
    optimal_swd_beta = df_value.loc['SWD-BoN'].idxmax()
    optimal_skl_beta = df_value.loc['SKL-Bon'].idxmax()
    optimal_values.append({
        "proxy": proxy,
        "optimal_wd_beta": optimal_wd_beta,
        "optimal_wd_value": optimal_wd_value,
        "optimal_kl_beta": optimal_kl_beta,
        "optimal_kl_value": optimal_kl_value,
        'optimal_l_beta':optimal_l_beta,
        'optimal_l_value':optimal_l_value,
        'optimal_swd_beta':optimal_swd_beta,
        'optimal_swd_value':optimal_swd_value,
        'optimal_skl_beta':optimal_skl_beta,
        'optimal_skl_value':optimal_skl_value
    })


#evaluate test dataset
dataset = 'hh-helpful'
# dataset = 'alpaca'
# dataset = 'hh-harmless'
sns.set_style("darkgrid", {'grid.linestyle': '--'})
sns.set_context("paper", 3.0, {"lines.linewidth": 4})
fig, axes = plt.subplots(2, 3, figsize=(30, 18))
plt.subplots_adjust(hspace=0.3)
axes = axes.flatten()
proxy_model = ["OASST", "SHP-Large", "SHP-XL", 'PairRM', 'RM-Mistral-7B']
for idx, opt in enumerate(optimal_values):
    proxy = opt['proxy']
    opt_k_beta = opt['optimal_kl_beta']
    opt_wd_beta = opt['optimal_wd_beta']
    opt_l_beta = opt['optimal_l_beta']
    opt_swd_beta = opt['optimal_swd_beta']
    opt_skl_beta = opt['optimal_skl_beta']
    
    df_path = f"./Compare_Result/{dataset}/{model},g-{gold},p-{proxy}.csv"
    df_ = pd.read_csv(df_path)

    df_test_value = pd.DataFrame(index=['WD-BoN', 'KL-BoN', 'random', 'MBR','l-BoN','SWD-BoN','SKL-BoN'], columns=betas)
    for j, beta in enumerate(betas):
        
        wd_win_rate = (df_.iloc[:, 0] <df_.iloc[:, j + 2]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, j + 2]).mean()
        kl_win_rate = (df_.iloc[:, 0] < df_.iloc[:, j + 2 + len(betas)]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, j + 2 + len(betas)]).mean()
        mbr_win_rate = (df_.iloc[:, 0] < df_.iloc[:, 1]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, 1]).mean()
        random_win_rate = (df_.iloc[:, 0] < df_.iloc[:, -1]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, -1]).mean()
        swd_win_rate = (df_.iloc[:, 0] < df_.iloc[:, j + 2 + len(betas) + len(betas)]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, j + 2 + len(betas) + len(betas)]).mean()
        skl_win_rate = (df_.iloc[:, 0] < df_.iloc[:, j + 2 + len(betas) + len(betas) + len(betas)]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, j + 2 + len(betas) + len(betas) + len(betas)]).mean()
        l_win_rate = (df_.iloc[:, 0] < df_.iloc[:, j + 2 + len(betas) + len(betas) + len(betas) + len(betas)]).mean() + 0.5 * (df_.iloc[:, 0] == df_.iloc[:, j + 2 + len(betas) + len(betas) + len(betas) + len(betas)]).mean()
        
        df_test_value.at['WD-BoN', beta] = wd_win_rate * 100
        df_test_value.at['KL-BoN', beta] = kl_win_rate * 100
        df_test_value.at['l-BoN', beta] = l_win_rate * 100
        df_test_value.at['MBR', beta] = mbr_win_rate * 100
        df_test_value.at['random', beta] = random_win_rate * 100
        df_test_value.at['SWD-BoN', beta] = swd_win_rate * 100
        df_test_value.at['SKL-BoN', beta] = skl_win_rate * 100
        
    ax = axes[idx]
    ax.plot(betas, df_test_value.loc['WD-BoN'], label='$\mathrm{{RBoN}}_{{\mathrm{{WD}}}}$', color='blue')
    ax.plot(betas, df_test_value.loc['KL-BoN'], label='$\mathrm{{RBoN}}_{{\mathrm{{KL}}}}$', color='red')
    ax.plot(betas, df_test_value.loc['l-BoN'], label='$\mathrm{{RBoN}}_{{\mathrm{{L}}}}$', color='yellow')
    ax.plot(betas, df_test_value.loc['MBR'], label='MBR', color='purple')
    ax.plot(betas, df_test_value.loc['random'], label='Random', color='brown')
    ax.plot(betas, df_test_value.loc['SWD-BoN'], label='$\mathrm{{RBoN}}_{{\mathrm{{SWD}}}}$', color='black')
    ax.plot(betas, df_test_value.loc['SKL-BoN'], label='$\mathrm{{RBoN}}_{{\mathrm{{SKL}}}}$', color='green')
    
    # Highlighting the optimal points
    ax.plot(opt_wd_beta, df_test_value.at['WD-BoN', opt_wd_beta], 'x', label=f'Optimal Beta ($\mathrm{{RBoN}}_{{\mathrm{{WD}}}}$) : {opt_wd_beta}', markersize=15,markeredgewidth=10, color='blue')
    ax.plot(opt_k_beta, df_test_value.at["KL-BoN", opt_k_beta], 'x', label=f'Optimal Beta ($\mathrm{{RBoN}}_{{\mathrm{{KL}}}}$) : {opt_k_beta}', markersize=15,markeredgewidth=10, color='red')
    ax.plot(opt_l_beta, df_test_value.at["l-BoN", opt_l_beta], 'x', label=f'Optimal Beta ($\mathrm{{RBoN}}_{{\mathrm{{L}}}}$) : {opt_l_beta}', markersize=15,markeredgewidth=10, color='yellow')
    ax.plot(opt_swd_beta, df_test_value.at["SWD-BoN", opt_swd_beta], 'x', label=f'Optimal Beta ($\mathrm{{RBoN}}_{{\mathrm{{SWD}}}}$) : {opt_swd_beta}', markersize=15,markeredgewidth=10, color='black')
    ax.plot(opt_skl_beta, df_test_value.at["SKL-BoN", opt_skl_beta], 'x', label=f'Optimal Beta ($\mathrm{{RBoN}}_{{\mathrm{{SKL}}}}$) : {opt_skl_beta}', markersize=15,markeredgewidth=10,color='green')
    
    ax.set_xscale('log')
    axes[5].axis('off')
    ax.set_xlabel('Beta', fontsize=30)
    ax.set_title(f'{proxy_model[idx]}', fontsize=30)
    ax.set_ylabel('Win Rate (%)', fontsize=30)
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.show()
